In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import wfdb

import os
import tqdm

# Infarction Stadium Labels
Infarction Stadium details how damaged the heart tissue is.
* I used the methodology described in this paper to distill the labels: https://arxiv.org/pdf/2306.15681

# Objective
Given a 12 lead ECG system, classify the infarction stadium level.
* Noise may or may not be present in the signal

# Strategy
* First, train a neural network (NN) to identify the four types of noise ['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems']
* Next, train a neural network to identify the infarction stadium (IS) using the 12 lead signal information with an additional layer than indicates what type of noise is in the signal for each lead.
* In Production, each lead is passed through the noise detection NN to detect noise. That information is then passed to the IS neural network with the signals to make a classification.

In [ ]:
root_directory_path = '/home/complukter/Desktop/PTBXL-data'
sig_database = pd.read_csv(os.path.join(root_directory_path, 'ptbxl_database.csv'), index_col='ecg_id')
sig_database.columns

Remove the stuff we don't need

In [ ]:
sig_database = sig_database.drop(
    ['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site', 'device', 'recording_date', 'validated_by', 'second_opinion', 'initial_autogenerated_report', 'validated_by_human','extra_beats', 'pacemaker'],
    axis=1
)
print(sig_database.columns)

# Looking at one Signal
* We will use the 100 hz sample

In [ ]:
def load_one_sample(path):
    data = wfdb.rdsamp(path)
    signals = np.asmatrix(data[0])
    meta = data[1]
    return signals, meta

one_sample = sig_database.iloc[0]
file_path = one_sample['filename_lr']
signals, meta = load_one_sample(os.path.join(root_directory_path, file_path))
print(signals, meta)
signal = signals.flatten().tolist()[0]
print(signal)
plt.plot(range(len(signal)), signal)
plt.show()

# Noise Identification Dataset
The issue with this dataset is that there is lots of noise in the signals. Thankfully, the 4 types of noise have been tagged by the lead. I want to clean up this dataset and create a noise classifier, however, there is a lot of garbage labelings. The goal is to first make a pipeline that can parse the strings and map them to the leads, and for the examples where it's not clear,  look at them individually. Additionally, tag the clean ones as clean.

Assumptions we are making:
- the ';' is replacable with a ',' not a '-'. Meaning it does not represent a range.
- Any other strings other than lead names, 'nan' or 'alles' will be rejected from the dataset
- lead order in the 'lead_table' comes from the signal meta data from one of the signal samples

In [ ]:
lead_table = ['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6']
error_message = 'erroneous'
clean_message = 'clean'

In [ ]:
def filter_lead_symbols(lead_string):

    # if the string is empty, we want to tag it as clean
    lead_string = str(lead_string).lower()

    if lead_string == 'nan':
        return clean_message
    
    lead_string = lead_string.replace(';', ',')
        
    strings = [ s.strip().replace(' ', '') for s in str(lead_string).split(',') if s.replace(' ', '') != '']
    final_symbols = []
    for s in strings:

        # All values
        if s == 'alles':
            return lead_table
        
        # Dash Present
        elif '-' in s:
            symbol_range = s.split('-')

            # check if the second value is just a number, if so, add a 'v' to the front of it
            if symbol_range[-1].isdigit():
                symbol_range[-1] = 'v' + symbol_range[-1]

            # Check if the range is possible, if not, then it is garbage
            if symbol_range[0] in lead_table and symbol_range[-1] in lead_table:
                start_index, end_index = lead_table.index(symbol_range[0]), lead_table.index(symbol_range[-1])
                symbols = lead_table[start_index: end_index + 1]
                final_symbols.extend(symbols)

            ####### Garbage
            else:
                return error_message
            
        # no dash
        else:
            if s in lead_table:
                final_symbols.append(s)

            ####### Garbage
            else:
                return error_message

    return final_symbols if len(final_symbols) else error_message

In [ ]:
# Remove rows that were erroneous
sig_database[['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems']] = sig_database[['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems']].map(lambda x: filter_lead_symbols(x))
mask = sig_database[['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems']].map(lambda x: error_message in str(x)).any(axis=1)
sig_database = sig_database[~mask]
len(sig_database)

# At this point...
We have parsed the lead information for the four types of noise present in the signals and have removed the unparsable examples from the dataset. Now we will create a dataset where each example is a signal and the label is 5 element array indicating the presence of the noise, for example:

# Format ['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'clean'] could translate to [0, 1, 1, 0, 0] to indicate 'baseline_drift' and 'static noise'
- If there is any noise whatsoever, the clean flag cannot be set.

The way the signal data is stored is in a (10 sec * sample_rate) length array of length 12 arrays, where each index in the 12 value array refers to a specific lead. We first need to flip the array to be oriented the other way, then create the labels and export the dataset.

In [ ]:
def load_and_transform(df):
    file_path = df['filename_lr']
    signals, meta = load_one_sample(os.path.join(root_directory_path, file_path))
    signals = signals.T # transpose to get each individual signal on it's own row
    noise_types = ['baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems', 'clean'] # The last column is Clean

    classifications = np.zeros((signals.shape[0], len(noise_types)))    # This will store the classifications data
    for noise_index, type in enumerate(noise_types):
        if type == 'clean':
            break
        # Get the indexes of the signals that contain this type of noise using the lead code
        codes_for_noise_type = df[type]

        # No Leads have noise
        if codes_for_noise_type != 'clean':
            for code in codes_for_noise_type:
                lead_index = lead_table.index(code)
                classifications[lead_index][noise_index] = 1

    # If no noise it present in the column, mark the clean label as true
    for lead_index in range(classifications.shape[0]):
        if np.sum(classifications[lead_index: -1]) == 0.0:
            classifications[lead_index][-1] = 1

    # Export the signal, label matchings
    rows = []
    for lead_index in range(signals.shape[0]):
        rows.append({'signal': signals[lead_index].tolist()[0], 'label' : classifications[lead_index]})

    return pd.DataFrame(rows)

load_and_transform(sig_database.iloc[0])


In [ ]:
# Apply this transform to the entire dataset, producing a new dataframe, export it to a file
# Initialize an empty list to collect all rows

with open('noisey_signal_dataset.csv', 'w') as output_file:

    # Use tqdm to monitor the progress
    for index, row in tqdm.tqdm(sig_database.iterrows(), total=sig_database.shape[0], desc="Processing rows"):
        formatted_data = load_and_transform(row)

        formatted_data.columns = ['signal', 'classification']
        formatted_data.to_csv(output_file, mode='a', header=(index == 0), index=False)

        # Drop the row to free up space
        sig_database.drop(index, inplace=True)
        